<a href="https://colab.research.google.com/github/sameepshrestha/Mission-Chatbot/blob/main/lstm_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM
import csv 
import pandas as pd 
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.activations import  sigmoid, tanh
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
import numpy as np 
# from sklearn.preprocessing import train_test_split
from tqdm import tqdm

In [2]:
# encoder and decoder both input_shape = [batch ,time_step,feature] 
batch_size = 64
time_step = 31
epochs = 200
vector_shape  = 300  #the loss we wil be using is the cosine 
#the ct and the ht learned from the encoder as a context is passed to the decoder 
data_set = pd.read_csv('/content/drive/MyDrive/chat bots/trainable_dataset.csv')
count = len(data_set)
count=25000

# New Section

In [3]:

vocab = {}
vocab_reverse = {}
with open ('/content/drive/MyDrive/chat bots/metadata (1).tsv') as f:
    for i,row in enumerate(csv.reader(f )):
        vocab[row[0]] = i
        vocab_reverse[i] = row[0]

In [4]:

vectors = np.zeros([total_size,vector_shape])
with open ('/content/drive/MyDrive/chat bots/vectors (1).tsv') as f:
    for i,row in enumerate(csv.reader(f,delimiter="\t")):
        vectors[i,:] = [np.array(float(x)) for x in row]
print(vectors.shape)

NameError: ignored

In [ ]:
# well this is a mess no stacked as one opposes stack and other support it but i think the attention model 
#without the stack should also be the best option >next target if this becomes some what sucessful
sequence_length =  30
encoder_inputs = Input(shape=( sequence_length,vector_shape))
encoder = LSTM(300, return_sequences=False, return_state=True)
encoder_outputs, stateh, statec = encoder(encoder_inputs)
encoder_states = [stateh,statec]
decoder_inputs = Input(shape= (None,vector_shape))
decoder = LSTM(300,return_sequences=True, return_state=True )
decoder_outputs,_ ,_  = decoder(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(vector_shape, activation = tanh)
decoder_outputs = decoder_dense(decoder_outputs)
seq2seq_model = Model([encoder_inputs,decoder_inputs],decoder_outputs)
seq2seq_model.summary()

In [ ]:
cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)
def loss_function(actual, output):
    loss = 1-cosine_loss(actual, output)
    return loss
    

In [ ]:
opt = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
seq2seq_model.compile(optimizer=opt, loss = loss_function)

In [ ]:
encoder_input = np.zeros([count,sequence_length,300])
decoder_input = np.zeros([count,sequence_length,300])
decoder_target = np.zeros([count,sequence_length,300])
for i, (data1, data2) in enumerate(zip(data_set['clean_message'][0:count],data_set['clean_reply'][0:count])):
    seq1 = [vocab[word] for word in reversed(data1.split())]#from paper http://arxiv.org/abs/1409.3215v3
    #havent understood about the <EOF> and the vector to represent that so we will think that '' is the EOF for now 
    seq2 = [vocab[words] for words in data1.split()]
    for t in range(sequence_length) :
        if t < sequence_length-len(seq1)-1 or t==sequence_length-1:
            encoder_input[i,t] = np.array(vectors[0])
        else :
            encoder_input[i,t] = np.array(vectors[seq1[t-(sequence_length-len(seq1))]])
        decoder_input[i,0] =  np.array(vectors[0])
        if t < len(seq2):
            decoder_target[i,t] = np.array(vectors[seq2[t]])
        else :
            decoder_target[i,t] = np.array(vectors[0])
decoder_input[:,1:sequence_length] = decoder_target[:,0:sequence_length-1]      
    # for words in data2.split()

In [ ]:
print(encoder_input[0])
print(decoder_input[0])
print(decoder_target[0])
encoder_input.shape

In [ ]:
seq2seq_model.fit([encoder_input,decoder_input],decoder_target,batch_size=16,epochs=epochs,validation_split=.1)
print(x.shape)

In [ ]:
import re
def remove(message):
    pure = ''
    for letter in message:
        if letter in 'abcdefghijklmnopqrstuvwxyz' or letter == ' ':
              pure = pure + letter
    # print(pure)
    return pure
        
def remove_haha(message):

    if re.search('haha',message):
            word = 'haha'
    else :
            word = message
    return word
def vocab_change(message):
    token = []
    for words in message:
        if words in vocab.keys():
            token.append(vocab[words])
        else :
            token.append(1)
    return token 
def prediction_pattern(message):
    message = remove(message)
    tokens = [remove_haha(word) for word in reversed(message.split())]
    seq1 = vocab_change(tokens)
    encoder_inp = np.zeros([1,30,vector_shape])
    for t in range(sequence_length) :
        if t < sequence_length-len(seq1)-1 or t==sequence_length-1:
            encoder_inp[0,t] = vectors[0]
        else :
            encoder_inp[0,t] = vectors[seq1[t-(sequence_length-len(seq1))]]
    return encoder_inp
prediction_pattern(('oe k gardai xas'))

In [ ]:
encoder_model =Model(encoder_input,encoder_states) #thans for https://medium.com/deep-learning-with-keras/seq2seq-part-d-encoder-decoder-with-teacher-forcing-18a3a09a096
#took some time to understand the everything but well written and well implemented
decoder_state_input_h = Input(shape=(vector_shape,))
decoder_state_input_c = Input(shape=(vector_shape,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
def decode(sentences):
    encoder_input = prediction_pattern(sentences.lower())
    states_value = encoder_model.predict(encoder_input)
    target_seq = np.zeros([1,1,vector_shape])
    target_seq[0,0,vector_shape] = vectors[0]
    print(target_seq)
    stop_condition = True 
    while not stop_condition:

        # in a loop
        # decode the input to a token/output prediction + required states for context vector
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # convert the token/output prediction to a token/output
        sampled_token_index = np.argmax(vectors*np.squeeze(output_tokens))
        print(vocab_reverse[sampled_token_index])
        # add the predicted token/output to output sequence
        

        # Exit condition: either hit max length
        # or find stop character.
        decoded_seq.append(vocab_reverse[sampled_token_index)
        if (sampled_token_index == 0 or
           len(decoded_seq) == n_timesteps_in):
            stop_condition = True

        # Update the input target sequence (of length 1) 
        # with the predicted token/output 
        target_seq = np.zeros((1, 1, n_features))
        target_seq[0, 0, sampled_token_index] = vectors['sampled_token_index']

        # Update input states (context vector) 
        # with the ouputed states
        states_value = [h, c]

        # loop back.....
        
    # when loop exists return the output sequence
    return decoded_seq


decode('oe k gardai xas')

In [ ]:
target_seq = np.zeros([1,1,vector_shape])

In [ ]:
print(vectors.shape)
target = target_seq.reshape([300])
np.dot(vectors,target)